In [2]:
import pandas as pd
import torch
from torch.utils import data
from d2l import torch as d2l

In [5]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = d2l.synthetic_data(true_w,true_b,1000)   
# 之前的生成数据的方式一个封装，相当于围绕真实的w，b生成一组扰动的数据，平时也可以用到

In [15]:
# 生成线性回归的扰动数据的情况
# fea = features.numpy()
# Data = pd.DataFrame(fea.tolist())
# Data.to_excel("从tensor转化为excel.xlsx",index=False)